In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [2]:
import pandas as pd
import requests
from io import BytesIO

In [3]:
from data_management.models import *

In [4]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [5]:
class DataExtractSave():
    def __init__(self):
        self.left = []
        self.top = []
        self.right = []
        self.bottom = []
        self.origin_url = []
        self.width = []
        self.height = []
    
    def get_image_size(self, resp):
        from PIL import Image
        byteImgIO = BytesIO()
        byteImg = Image.open(BytesIO(resp.content))
        byteImg.save(byteImgIO, "JPEG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        dataBytesIO = BytesIO(byteImg)
        image = Image.open(dataBytesIO)
        image = image.convert('RGB')
        width, height = image.size
        return width, height
    
    def get_data_and_append(self, obj):
        if obj.origin_source.image:
            url = obj.origin_source.image.url
        else:
            url = obj.origin_source.s3_image_url
        self.origin_url.append(url)
        resp = requests.get(url)
        w, h = self.get_image_size(resp)
        self.width.append(w)
        self.height.append(h)
        self.left.append(obj.left*w)
        self.top.append(obj.top*h)
        self.right.append(obj.right*w)
        self.bottom.append(obj.bottom*h)
    
    def save_to_csv(self,  filename):
        dic = {}
        dic['left'] = self.left
        dic['top'] = self.top
        dic['right'] = self.right
        dic['bottom'] = self.bottom
        dic['origin_url'] = self.origin_url
        dic['width'] = self.width
        dic['height'] = self.height
    
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('downloaded')
        
    def save_data_from_cropped(self, filename, n):
        print(n)
        N = int(n)
        if not n:
            queryset = CroppedImage.objects.filter(origin_source__image__isnull=False)
        else:
            queryset = CroppedImage.objects.filter(origin_source__image__isnull=False)[:N]
    
        for c in queryset:
            print(c)
            self.get_data_and_append(c)
            print(self.left)
        self.save_to_csv(filename)
        
    def save_data_from_s3_temp(self, filename, n):
        print(n)
        N = int(n)
        if not n:
            queryset = CroppedImage.objects.filter(origin_source__image="")
        else:
            queryset = CroppedImage.objects.filter(origin_source__image="")[:N]
            print(queryset)
    
        for c in queryset:
            print(c)
            self.get_data_and_append(c)
            print(self.left)
        self.save_to_csv(filename)

In [6]:
downloadmanager= DataExtractSave()

In [7]:
# save_data_from_cropped : 크로링 박싱후 저장한 데이터 csv로 저장
# save_data_from_cropped('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_cropped("crop_download_test.csv","")

In [8]:
downloadmanager.save_data_from_cropped("crop_download_test.csv","3")

3
CroppedImage object (178)
[Decimal('255.8976')]
CroppedImage object (179)
[Decimal('255.8976'), Decimal('77.3718')]
CroppedImage object (180)
[Decimal('255.8976'), Decimal('77.3718'), Decimal('75.3033')]
crop_download_test.csv
downloaded


In [9]:
# save_data_from_s3_temp : s3업로드 파일 박싱후 저장한 데이터 csv로 저장
# save_data_from_s3_temp('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_s3_temp("s3upload_download_test.csv","")

In [10]:
downloadmanager.save_data_from_s3_temp("s3upload_download_test.csv","3")

3
<QuerySet [<CroppedImage: CroppedImage object (5736)>, <CroppedImage: CroppedImage object (5737)>, <CroppedImage: CroppedImage object (5738)>]>
CroppedImage object (5736)
[Decimal('255.8976'), Decimal('77.3718'), Decimal('75.3033'), Decimal('8.9892')]
CroppedImage object (5737)
[Decimal('255.8976'), Decimal('77.3718'), Decimal('75.3033'), Decimal('8.9892'), Decimal('6.9916')]
CroppedImage object (5738)
[Decimal('255.8976'), Decimal('77.3718'), Decimal('75.3033'), Decimal('8.9892'), Decimal('6.9916'), Decimal('0.0000')]
s3upload_download_test.csv
downloaded
